In [ ]:
myseed = 123
Nb_classes = 27

from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                   rescale = 1./255)

val_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                   rescale = 1./255)

test_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,
                                   rescale = 1./255)


batch_size = 64


train_dataset = train_datagen.flow_from_directory( directory = '../Splitted_datasets/Images_224px/Train_subset',
                                                   target_size = (224, 224),
                                                   color_mode = 'rgb',
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle = False,
                                                   seed = myseed)

val_dataset = val_datagen.flow_from_directory( directory = '../Splitted_datasets/Images_224px/Validation_subset',
                                                   target_size = (224, 224),
                                                   color_mode = 'rgb',
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle = False,
                                                   seed = myseed)

test_dataset = test_datagen.flow_from_directory( directory = '../Splitted_datasets/Images_224px/Test_subset',
                                                   target_size = (224, 224),
                                                   color_mode = 'rgb',
                                                   batch_size = 1,
                                                   class_mode = None,
                                                   shuffle = False,
                                                   seed = myseed)

### Found 54345 images belonging to 27 classes.
### Found 13587 images belonging to 27 classes.
### Found 16984 images belonging to 1 classes.


from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential

# get pre-rained VGG16 layers
base_model = VGG16(weights = 'imagenet', include_top = False)

# freeze layers
for layer in base_model.layers:
    layer.trainable = False

# new sequential model
VGG_model = Sequential()
VGG_model.add(base_model)
VGG_model.add(GlobalAveragePooling2D())
VGG_model.add( Flatten() )
VGG_model.add( Dense(units = 512, activation = 'relu') )
VGG_model.add( Dropout(rate = 0.7) )
VGG_model.add( Dense(units = 128, activation = 'relu') )
VGG_model.add( Dense(units = Nb_classes, activation='softmax') )


# checkpoint callbacks
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = './tmp_checkpoint/img_base_model_vgg'
checkpoint = ModelCheckpoint(
                            filepath=checkpoint_filepath,
                            verbose = 1,
                            save_weights_only=True,
                            monitor='val_accuracy',
                            mode='max',
                            save_best_only=True)

# reduce lr on plateau callback
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_plateau = ReduceLROnPlateau(monitor = 'val_loss',
                               patience=3,
                               factor=0.5,
                               verbose=2,
                               mode='min',
                               min_lr = 1e-10)

Nb_epochs = 21
lr_0 = 1e-3

# compile model
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate = lr_0)

VGG_model.compile(loss = 'categorical_crossentropy',  # targets fromat is class numbers [2,3,22,..] from folder names.
                  optimizer = optimizer,
                  metrics = ['accuracy'])

# train model
STEP_SIZE_TRAIN=train_dataset.n//train_dataset.batch_size    ### 849
STEP_SIZE_VALID=val_dataset.n//val_dataset.batch_size        ### 212

training_history = VGG_model.fit(train_dataset,
                                 steps_per_epoch=STEP_SIZE_TRAIN,
                                 validation_data = val_dataset,
                                 validation_steps=STEP_SIZE_VALID,
                                 epochs = Nb_epochs,
                                 callbacks = [
                                             checkpoint,
                                             lr_plateau
                                             ],
                                 verbose = 1)


### progress ofr the first two epochs:
### Epoch 1/21
### 849/849 [==============================] - ETA: 0s - loss: 2.8980 - accuracy: 0.1618
### Epoch 1: val_accuracy improved from -inf to 0.28744, saving model to ./tmp_checkpoint\img_base_model_vgg
### 849/849 [==============================] - 5998s 7s/step - loss: 2.8980 - accuracy: 0.1618 - val_loss: 2.4921 - val_accuracy: 0.2874 - lr: 0.0010
### Epoch 2/21
### 849/849 [==============================] - ETA: 0s - loss: 2.5013 - accuracy: 0.2619
### Epoch 2: val_accuracy improved from 0.28744 to 0.30557, saving model to ./tmp_checkpoint\img_base_model_vgg
### 849/849 [==============================] - 5741s 7s/step - loss: 2.5013 - accuracy: 0.2619 - val_loss: 2.3451 - val_accuracy: 0.3056 - lr: 0.0010

Found 54345 images belonging to 27 classes.
Found 13587 images belonging to 27 classes.
Found 16984 images belonging to 1 classes.
Epoch 1/21
  5/849 [..............................] - ETA: 1:08:12 - loss: 3.5304 - accuracy: 0.0656 